In [1]:
import scipy as sp 
import numpy as np 
import pandas as pd
from FileExporter import create_download_link

In [15]:
from sklearn.datasets import load_diabetes
import random, string

X, y = load_diabetes(return_X_y=True)

print(X.shape)
print(y.shape)

''.join(random.choices(string.ascii_uppercase + string.digits, k=X.shape[0]))

def id_generator(size=10, chars=string.ascii_uppercase + string.digits):
    MadeUp_ProtNames, MadeUp_peptide_phosphosite = [], []
    for i in range(X.shape[0]):
        MadeUp_ProtNames.append(''.join(random.choice(chars) for _ in range(size)))
        MadeUp_peptide_phosphosite.append(''.join(random.choice(chars) for _ in range(size)))
    return pd.Series(MadeUp_ProtNames), pd.Series(MadeUp_peptide_phosphosite)


X = pd.DataFrame(X)

frame = [MadeUp_peptide_phosphosite, MadeUp_ProtNames, X]
X = pd.concat(frame, axis = 1)

(442, 10)
(442,)


In [ ]:
A = pd.read_csv('./msresist/data/20180817_JG_AM_TMT10plex_R1_psms.csv', header=0)
B = pd.read_csv('./msresist/data/20190214_JG_AM_PC9_AXL_TMT10_AC28_R2_PSMs.csv', header=0)
C = pd.read_csv('./msresist/data/CombinedBR3_TR1&2.csv', header=0)

#Data
a, b, c = A.iloc[:, 2:12], B.iloc[:, 2:12], C.iloc[:, 2:12]
t = list(a.columns)

#Peptide Sequences
Aseqs, Bseqs, Cseqs = A.iloc[:, 0], B.iloc[:, 0], C.iloc[:, 0]

#Protein Names 
Anames, Bnames, Cnames = [], [], []
x = list(map(lambda  v : Anames.append(v.split("OS")[0]), A.iloc[:,1]))
y = list(map(lambda  v : Bnames.append(v.split("OS")[0]), B.iloc[:,1]))
z = list(map(lambda  v : Cnames.append(v.split("OS")[0]), C.iloc[:,1]))

First I'm concatenating all 3 BR, then I'm counting the number of repeated rows in the data set. Surprisingly none of the peptides were present in all 3 MS experiments, whereas 286 peptides appear in 2 of the experiments. I then compute the mean of every duplicate.  

In [ ]:
#Concatenate 3 BR
frames = [A,B,C]
ABC = pd.concat(frames)
print("shape of concatenated matrix:", ABC.shape)

#count number of duplicates across data sets
dups = ABC.pivot_table(index=['Master Protein Descriptions', 'peptide-phosphosite'], aggfunc = "size")
my_dict = {i:list(dups).count(i) for i in list(dups)}
print("total number of duplicates:", my_dict)

#compute mean across duplicates 
func = {}
for i in t:
    func[i] = np.mean
ABC_avg = pd.pivot_table(ABC, values = t, index = ['Master Protein Descriptions', 'peptide-phosphosite'], aggfunc = func)

dups2 = ABC_avg.pivot_table(index=['Master Protein Descriptions', 'peptide-phosphosite'], aggfunc = "size")
print("shape of averaged matrix:", ABC_avg.shape)
create_download_link(pd.DataFrame(ABC_avg), "Combined_MS_replicates_initial")

Manually copied/pasted a specific phosphopeptide 6 times in excel (df 'ABC_foo'), and the method correctly displayed them, reinforcing the reliabability of the results above

In [ ]:
D = pd.read_csv('./msresist/data/ABC_foo.csv', header=0)
dups = D.pivot_table(index=['Master Protein Descriptions', 'peptide-phosphosite'], aggfunc = "size")
create_download_link(pd.DataFrame(dups), "D_dups")